Connect to FRED via the fredr package - Short for Federal Reserve Economic Data, FRED is an online database consisting of hundreds of thousands of economic data time series from scores of national, international, public, and private sources. FRED, created and maintained by the Research Department at the Federal Reserve Bank of St. Louis, goes far beyond simply providing data: It combines data with a powerful mix of tools that help the user understand, interact with, display, and disseminate the data. In essence, FRED helps users tell their data stories. The purpose of this article is to guide the potential (or current) FRED user through the various aspects and tools of the database.

This is likely the easiest way to get data from U.S. statistical agencies like the Census and the BLS

In order to get this code to work you must obtain an api key from FRED: https://fredaccount.stlouisfed.org/apikeys

[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/MarjorieRWillner/DisasterHack/blob/main/econ_data_extract_r.ipynb)


In [ ]:
# This code will gather the population of each US state over time

# install.packages("fredr")
# install.packages("tidyverse")
library(fredr)
library(tidyverse)

# your api key goes here
fredr_set_key("")

# the fredr series id we want is DCPOP, FLPOP, VAPOP, MDPOP, etc. We will create
# a list with all the states in the format of [ST]POP.
state_id <- c(map_chr(state.abb, ~ paste0(.x, "POP")), "DCPOP")

# We can now map over state_id and get the population for each state using the
# fredr() function
state_df <- map_dfr(
  state_id,
  ~ fredr(
    series_id = .x, 
    observation_start = as.Date("2020-01-01"), 
    observation_end = as.Date("2021-01-01")
    )
  )

glimpse(state_df)

Warning message:
“Failed to locate timezone database”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



Rows: 102
Columns: 5
$ date           <date> 2020-01-01, 2021-01-01, 2020-01-01, 2021-01-01, 2020-0…
$ series_id      <chr> "ALPOP", "ALPOP", "AKPOP", "AKPOP", "AZPOP", "AZPOP", "…
$ value          <dbl> 5024.803, 5039.877, 732.441, 732.673, 7177.986, 7276.31…
$ realtime_start <date> 2022-01-20, 2022-01-20, 2022-01-20, 2022-01-20, 2022-0…
$ realtime_end   <date> 2022-01-20, 2022-01-20, 2022-01-20, 2022-01-20, 2022-0…


Connect to Census via the tidycensus package - tidycensus is an R package that allows users to interface with a select number of the US Census Bureau’s data APIs and return tidyverse-ready data frames, optionally with simple feature geometry included.

Accessing the Census directly allows for retrievel of ACS microdata files so that we can combine in a way that makes sense for our project.



To get started working with tidycensus, users should load the package along with the tidyverse package, and set their Census API key. A key can be obtained from http://api.census.gov/data/key_signup.html.


NOTE: To install tidycensus in sagemake you need to run the following code in the terminal 

"conda install -n R -c conda-forge r-tidycensus"

Reference: https://aws.amazon.com/blogs/machine-learning/creating-a-persistent-custom-r-environment-for-amazon-sagemaker/

__________________________________________________________________________________________________________________________________

get_pums() returns some technical variables by default without the user needing to request them specifically. These include:

- SERIALNO: a serial number that uniquely identifies households in the sample;
- SPORDER: the order of the person in the household; when combined with SERIALNO, uniquely identifies a person;
- WGTP: the household weight;
- PWGTP: the person weight
- PUMA: Public Use Microdata Areas (PUMAs) are the smallest available geographies at which records are identifiable in the PUMS datasets. PUMAs are redrawn with each decennial US Census, and typically are home to 100,000-200,000 people. In large cities, a PUMA will represent a collection of nearby neighborhoods; in rural areas, it might represent several counties across a large area of a state.

It is a long-form dataset that organizes specific value codes by variable so you know what you can get. You'll use information in the var_code column to fetch variables, but pay attention to the var_label, val_code, val_label, and data_type columns.

In [ ]:
system(command = 'conda install -c r tidycensus --yes')

In [ ]:
# install.packages("tidycensus", repo="https://cran.rstudio.com/", dependencies = TRUE)
library("tidycensus")

# your api key goes here
census_api_key('', overwrite = TRUE, install = FALSE)

# we can get a list of all the variables that are available to us to choose from
pums_var <- pums_variables %>%
  filter(year == 2019, survey == "acs5") %>%
  arrange(var_label) 

glimpse(pums_var)

To install your API key for use in future sessions, run this function with `install = TRUE`.



Rows: 5,277
Columns: 12
$ survey     <chr> "acs5", "acs5", "acs5", "acs5", "acs5", "acs5", "acs5", "ac…
$ year       <chr> "2019", "2019", "2019", "2019", "2019", "2019", "2019", "20…
$ var_code   <chr> "ENG", "ENG", "ENG", "ENG", "ENG", "FENGP", "FENGP", "ACCES…
$ var_label  <chr> "Ability to speak English", "Ability to speak English", "Ab…
$ data_type  <chr> "chr", "chr", "chr", "chr", "chr", "chr", "chr", "chr", "ch…
$ level      <chr> "person", "person", "person", "person", "person", "person",…
$ val_min    <chr> "b", "1", "2", "3", "4", "0", "1", "b", "1", "2", "3", "b",…
$ val_max    <chr> "b", "1", "2", "3", "4", "0", "1", "b", "1", "2", "3", "b",…
$ val_label  <chr> "N/A (less than 5 years old/speaks only English)", "Very we…
$ recode     <lgl> TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE,…
$ val_length <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 7, 7, 7, 7, 7,…
$ val_na     <dbl> 0, 0, 0, 0, 0, NA, NA, 0, 0, 0, 0, -1, -1, -1, NA, NA, NA, …


In [ ]:
# use the get_pums() function to bring in all the microdata.
state_pums <- get_pums(
  variables = c(
    "ACCESS", "ADJINC", "AGEP", "CIT", "CITWP", "DECADE", "ELEP", "ENG", "FINCP", "GASP", "GRNTP", "GRPIP", "HHT", "HINCP", "HISPEED", "MIGSP", "MRGP", 
      "MV",  "NAICSP", "NATIVITY", "NOP", "NPF", "OCPIP", "OCCP", "PINCP", "PERNP", "POBP", "POVPIP", "PUMA", "REGION", "RNTP", "RT", "SCHL", "SEX", "ST",
      "TAXAMT", "TEN", "WAGP", "WATP", "WGTP", "YOEP"
  ),
  state = "all",
  survey = "acs1",
  recode = TRUE
)